In [1]:
import xgboost as xg
# import dask.dataframe as dd
import polars as pl
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
data = pl.read_csv('../data/preprocessed/engineered_training_set.csv')
data.describe()

statistic,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_brand_bool,position,srch_destination_id,srch_booking_window,srch_saturday_night_bool,orig_destination_distance,random_bool,click_bool,gross_bookings_usd,booking_bool,has_hist_starrating,has_hist_adr_usd,score1d2,normalized_prop_starrating,normalized_score1d2,normalized_prop_review_score,hotel_quality,normalized_hotel_quality,price_per_person,avg_price_per_day,ump,price_diff,starrating_diff,total_price,score2ma,promotion_count,promotion_any
str,f64,str,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,str,f64,f64
"""count""",4.953228e6,"""0""",4.953228e6,4.953228e6,"""251792""","""252914""",4.953228e6,4.953228e6,4.953228e6,4.953228e6,4.953228e6,4.953228e6,4.953228e6,3.348388e6,4.953228e6,4.953228e6,138275.0,4.953228e6,4.953228e6,4.953228e6,4.953228e6,4.953224e6,4.953224e6,4.953224e6,4.953224e6,4.953224e6,4.953228e6,4.953228e6,4.953228e6,"""252914""","""251792""",4.953228e6,"""0""",4.953228e6,4.953228e6
"""null_count""",0.0,"""4953228""",0.0,0.0,"""4701436""","""4700314""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.60484e6,0.0,0.0,4.814953e6,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,"""4700314""","""4701436""",0.0,"""4953228""",0.0,0.0
"""mean""",166367.108221,null,9.953445,175.369853,null,null,173.999283,70077.122916,0.634832,16.857549,14042.200748,37.473272,0.502294,1300.442127,0.295835,0.044751,385.632155,0.027916,1.0,1.0,0.0,1.3289e-9,0.0,5.8107e-9,2.1778e-9,0.57336,85.293109,92.34218,-0.981179,null,null,170.66645,null,6.277791,0.898359
"""std""",96110.062236,null,7.646976,65.904102,null,null,68.334449,40609.937868,0.481477,10.426479,8111.447094,51.993395,0.499995,2023.15977,0.456417,0.206756,804.686432,0.164733,0.0,0.0,0.0,0.970969,0.0,0.979434,0.484224,0.088648,82.613894,82.644123,107.787853,null,null,152.173464,null,5.896331,0.302176
"""min""",1.0,null,1.0,1.0,"""1.41""","""0.0""",1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-5.833333,0.0,-5.43742,-3.131872,0.0,0.0,0.0,-1998.16,"""-0.009999999999990905""","""-0.009999999999999787""",0.0,null,0.0,0.0
"""25%""",82936.0,null,5.0,100.0,null,null,100.0,35010.0,0.0,8.0,7101.0,4.0,0.0,139.76,0.0,0.0,123.99,0.0,1.0,1.0,0.0,-0.645497,0.0,-0.466997,-0.294509,0.5194433,40.0,40.25,-24.529985,null,null,89.0,null,2.0,1.0
"""50%""",166503.0,null,5.0,219.0,null,null,219.0,69630.0,1.0,16.0,13541.0,17.0,1.0,386.46,0.0,0.0,218.36,0.0,1.0,1.0,0.0,0.040506,0.0,0.222286,0.057451,0.583878,63.04,71.486667,17.834287,null,null,129.0,null,5.0,1.0
"""75%""",249721.0,null,14.0,219.0,null,null,219.0,105165.0,1.0,26.0,21084.0,48.0,1.0,1499.46,1.0,0.0,429.61,0.0,1.0,1.0,0.0,0.6561673,0.0,0.662652,0.345082,0.6365348,103.0,118.63,50.295562,null,null,199.64,null,9.0,1.0
"""max""",332785.0,null,34.0,231.0,"""5.0""","""999.81""",230.0,140821.0,1.0,40.0,28416.0,492.0,1.0,11666.64,1.0,1.0,159292.38,1.0,1.0,1.0,0.0,5.833334,0.0,5.294649,2.3304434,1.0,9150.0,1999.0,497.681251,"""997.0200000000001""","""5.0""",9150.0,null,35.0,1.0


In [3]:
data = data.sort(['srch_id', 'booking_bool', 'click_bool'], descending=[False, True, True])

In [4]:
print(data.shape)

(4958347, 54)


> CHANGE THE DATA TO ENGINEERED DATA, EVERYTHING SHOULD WORK

In [4]:
# Convert polars DataFrames to pandas DataFrames
data_pd = data.to_pandas()
# replace all NULL values with np.nan
data_pd = data_pd.replace('NULL', np.nan)
ranking_pd = data_pd[['srch_id', 'prop_id']]

# Convert object columns to appropriate data types
object_columns = data_pd.select_dtypes(include=['object']).columns
data_pd[object_columns] = data_pd[object_columns].apply(pd.to_numeric, errors='coerce')

# Split the data into features (X) and target (y)
X = data_pd.drop(['srch_id', 'prop_id'], axis=1)
y = ranking_pd['prop_id']


# Split the data into training and testing sets based on srch_id
srch_ids = data_pd['srch_id'].unique()
train_srch_ids, test_srch_ids = train_test_split(srch_ids, test_size=0.2, random_state=42)

# Create training and testing DataFrames
train_data = data_pd[data_pd['srch_id'].isin(train_srch_ids)]
test_data = data_pd[data_pd['srch_id'].isin(test_srch_ids)]

# Create training and testing ranking DataFrames
train_ranking = ranking_pd[ranking_pd['srch_id'].isin(train_srch_ids)]
test_ranking = ranking_pd[ranking_pd['srch_id'].isin(test_srch_ids)]


In [17]:


# Create XGBoost DMatrix objects for training and testing
train_dmatrix = xg.DMatrix(
    train_data.drop(['srch_id', 'booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1),
    label=train_data['prop_id'],
    group=train_data['srch_id'].value_counts().sort_index().values
)
test_dmatrix = xg.DMatrix(
    test_data.drop(['srch_id', 'booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1),
    label=test_data['prop_id'],
    group=test_data['srch_id'].value_counts().sort_index().values
)

# MAKE SURE TO USE FULL DATA SET IN THE END
full_dmatrix = xg.DMatrix(
    X.drop(['booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1),
    label=y,
    group=data_pd['srch_id'].value_counts().sort_index().values
)

# Set XGBoost parameters
params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg',
    'learning_rate': 0.1,
    'max_depth': 1,
    'subsample': 0.1,
    'colsample_bytree': 0.8,
    'seed': 42
}

# Train the XGBoost ranking model
model = xg.train(params, train_dmatrix, num_boost_round=100)



In [19]:
from sklearn.metrics import ndcg_score
# Make predictions on the test set
test_pred = model.predict(test_dmatrix)

# Evaluate the model
ndcg_scores = []
for srch_id in test_srch_ids[:5000]:
    # Get the data for the current srch_id
    srch_data = test_data[test_data['srch_id'] == srch_id]
    
    # Create DMatrix for the current srch_id
    srch_dmatrix = xg.DMatrix(srch_data.drop(['srch_id', 'booking_bool', 'gross_bookings_usd', 'position', 'click_bool'], axis=1))
    
    # Make predictions for the current srch_id
    srch_pred = model.predict(srch_dmatrix)
    
    # Get the corresponding prop_ids for the current srch_id
    srch_prop_ids = srch_data['prop_id']
    
    # Sort the prop_ids based on the predicted scores
    sorted_prop_ids = srch_prop_ids.iloc[np.argsort(srch_pred)[::-1]]
    
    # Get the ground truth rankings for the current srch_id
    test_ranking_query = test_ranking[test_ranking['srch_id'] == srch_id]['prop_id']
    
    # Calculate NDCG score for the current srch_id
    if len(test_ranking_query) < 2:
        continue
    ndcg = ndcg_score([test_ranking_query], [sorted_prop_ids], k=5)
    ndcg_scores.append(ndcg)

# Calculate the mean NDCG score
mean_ndcg = sum(ndcg_scores) / len(ndcg_scores)
print(f"Mean NDCG: {mean_ndcg:.4f}")

Mean NDCG: 0.4220


In [ ]:
model = xg.train(params, X, num_boost_round=100)

In [5]:
# save the model

model.save_model('models/model.json')
# load the model
model = xg.Booster()
model.load_model('models/model.json')

In [4]:

test_set = pl.read_csv('../data/raw/test_set_VU_DM.csv')
test_set = test_set.to_pandas()
test_set = test_set.replace('NULL', np.nan)

object_columns = test_set.select_dtypes(include=['object']).columns
test_set[object_columns] = test_set[object_columns].apply(pd.to_numeric, errors='coerce')

In [14]:
srch_ids = test_set['srch_id'].unique()

# Create an empty DataFrame to store the predictions
submission_df = pd.DataFrame(columns=['srch_id', 'prop_id'])

#  Unique srch_ids in the test set
srch_ids = test_set['srch_id'].unique()

# Pre-allocate a list to collect results
results = []

# Iterate over each srch_id and make predictions
for srch_id in tqdm(srch_ids):
    # Get the data for the current srch_id
    srch_data = test_set[test_set['srch_id'] == srch_id]

    # Create DMatrix for the current srch_id
    srch_dmatrix = xg.DMatrix(srch_data.drop(['srch_id'], axis=1))

    # Make predictions for the current srch_id
    srch_pred = model.predict(srch_dmatrix)

    # Get the corresponding prop_ids for the current srch_id
    srch_prop_ids = srch_data['prop_id'].values

    # Sort the prop_ids based on the predicted scores
    sorted_indices = np.argsort(srch_pred)[::-1]
    sorted_prop_ids = srch_prop_ids[sorted_indices]

    # Collect the results for the current srch_id
    results.append(pd.DataFrame({'srch_id': srch_id, 'prop_id': sorted_prop_ids}))

# Concatenate all results into a single DataFrame
submission_df = pd.concat(results, ignore_index=True)


# Check final submission DataFrame size
print(f"Expected number of entries: {len(test_set)}")
print(f"Actual number of entries: {len(submission_df)}")

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submit/submission.csv', index=False)

  0%|          | 0/199549 [00:00<?, ?it/s]

NameError: name 'test_data' is not defined